# 반복형, 반복자, 제너레이터

- 데이터를 처리할 때 반복은 기본
- 메모리에 다 들어가지 않는 데이터셋 검색할 때 항목들을 **느긋하게** 가져와야 함
    - 즉, 한번에 한번씩 필요할 때 가져와야 함
    - 반복자 패턴이 하는 일

**Note**
- 모든 제너레이터는 반복자
    - 제너레이터가 반복자 인터페이스를 완전히 구현하고 있기 때문
- 반복자는 [디자인 패턴] 에서 정의한 대로 컬렉션에서 항목을 가지고 옴
- 제너레이터는 '마술처럼' 항목 생성 가능
- 파이썬 커뮤니티에서는 **반복자** 와 **제너레이터**를 거의 동일시 

파이썬의 **컬렉션**은 모두 **반복형**, 다음과 같은 연산 지원하기 위해 내부적으로 반복자 사용
- for 루프
- 컬렉션형 생성과 확장
- 텍스트 파일을 한 줄씩 반복
- 지능형 리스트/딕셔너리/집합
- 튜플 언패킹
- 함수 호출 시 * 를 이용해서 실제 매개변수를 언패킹

## 14.1 Sentence 버전 #1 : 단어 시퀀스

#### 예제 14-1 sentence.py : 단어 시퀀스로서의 Sentence 클래스

In [4]:
import re
import reprlib

RE_WORD = re.compile('\w+')

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text) # 정규 표현식에서 매칭되는 중복되지 않는 문자열의 리스트를 반환
    
    def __getitem__(self, index):
        return self.words[index] # 주어진 index에 해당하는 단어 반환
    
    def __len__(self): # 시퀀스 프로토콜에 따르려면 필요, 반복형 객체에 이 메서드가 필요한 것은 아님
        return len(self.words)
    
    def __repr__(self):
        return 'Sentence(%s)' % reprlib.repr(self.text) # 매우 큰 데이터 구조체를 표현하는 문자열 축약해서 생성, 문자열을 30자로 제한

#### 14-2 Sentence 객체의 반복 테스트

In [5]:
s = Sentence('"The time has come," the  Walrus said,') # 문자열을 이용하여 Sentence 객체 생성
s # 출력한 메세지는 reprlib.repr()이 생성

Sentence('"The time ha... Walrus said,')

In [6]:
for word in s: # Sentence 객체는 반복할 수 있음
    print(word)

The
time
has
come
the
Walrus
said


In [7]:
list(s) # 반복할 수 있으므로 Sentece 객체는 리스트 혹은 다른 반복형을 생성하기 위한 입력으로 사용 가능

['The', 'time', 'has', 'come', 'the', 'Walrus', 'said']

<hr>

### 14.1.1 Sequemce가 반복 가능한 이유 : iter() 함수

- 파이썬 인터프리터가 x 객체를 반복해야 할 때는 언제나 iter(x)를 자동으로 호출
- iter() 내장함수는 다음 과정 수행
    1. 객체가 \__iter\__() 메서드를 구현하는지 확인, 이 메서드를 호출해서 반복자 가지고 옴
    2. \__iter\__()  메서드가 구현되어 있지 않지만 \__getitem\__() 이 구현되어 있다면, 파이썬은 인덱스 0에서 시작해서 항목을 순서대로 가져오는 반복자 생성
    3. 이 과정이 모두 실패하면 파이썬은 **TypeError ; 'C' object is not iterable** 발생

- 그렇기 때문에 모든 파이썬 시퀀스는 반복 가능
- 사실 표준 시퀀스는 \__iter\__() 메서드도 구현하고 있으므로 이 메서드도 구현

<br>


In [8]:
class Foo:
    def __iter__(self):
        pass
    

from collections import abc

print(issubclass(Foo, abc.Iterable))


f = Foo()
print(isinstance(f, abc.Iterable))

########################################################

print(issubclass(Sentence, abc.Iterable))


s = Sentence("e")
print(isinstance(s, abc.Iterable))

True
True
False
False


## 14.2 반복형과 반복자

**반복형**
- iter() 내장 함수가 반복자를 가져올 수 있는 모든 객체와 반복자를 반환하는 \__iter\__() 메서드를 구현하는 객체는 반복형
- 0에서 시작하는 인덱스를 받는 \__getitem\__() 메서드를 구현하는 객체인 시퀀스도 마찬가지

In [9]:
s = "ABC" # 'ABC' 문자열 반복형, 반복자 보이지 않지만 내부 어딘가 존재

it = iter(s) # 반복형에서 반복자 it을 생성

while True:
    try:
        print(next(it)) # 반복자에서 next를 계속 호출해서 다음 항목을 가져옴 
    except StopIteration: # 더 이상 항목이 없으면 예외 발생
        del it # it에 대한 참조 해제하여 반복자 객체 제겨
        break

A
B
C


**\__next\__()**
- 다음에 사용할 항목 반환
- 더 이상 항목이 남아있지 않으면 StopIteration 발생

<br>

**\__self\__()**
- self 반환
- for 루프 등 반복형이 필요한 곳에 반복자 사용할 수 있게 만듦

<img src="./images/3.png">

- Iterable과 Iterator ABC
- Iterable의 구상 서브 클래스의 \__iter\__() 메서드는 Iterator 객체를 생성하고 반환해야 함
- Iterator의 구상 서브 클래스는 \__next\__() 메서드를 구현해야 함
- Iterator.\__iter\__() 메서드는 self를 반환

#### 예제 14-3 abc.Iterator 클래스의 일부

In [14]:
class Iterator(Iterable):
    
    __slots__ = ()
    
    @abstractmethod
    def __next__(self):
        '''반복자에서 다음 항목을 반환, 항목이 소진되면 StopIteeration 예외 발생'''
        raise StopIteration
        
    def __iter__(self):
        return self
    
    @classmethod
    def __subclasshook__(cls, C):
        if cls is Iterator:
            if (any("__next__" in B.__dict__ for B in C.__mro__) and
                any("__iter__" in B.__dict__ for B in C.__mro__)):
                return True
            
        return NotImplemented

NameError: name 'Iterable' is not defined

<hr>

In [19]:
s3 = Sentence("Pig and Pepper") # 세 단어로 구성된 Sentece 객체 생성
it = iter(s3) # s3에서 반복자 가지고 옴
print(it)

print("-" * 30)

print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴
print(next(it)) # 다음단어 가지고 옴


------------------------------
Pig
and
Pepper


StopIteration: 

In [21]:
print(list(it)) # 소진 된 후에는 반복자 필요 없음
print(list(iter(s3))) # Sentence를 다시 반복하려면 생성자 새로 만들어야 함

[]
['Pig', 'and', 'Pepper']


<hr>

- 반복자가 필수적으로 구현해야 하는 메서드는 \__next\__()와 \__iter\__()
    - next()를 호출하고 StopIteration 예외를 잡는 방법 외에는 항목이 소진되었는지 확인할 방법이 없음
- 반복자는 "재설정" 할 수 없음
    - 다시 반복해야하면 처음 반복자를 생성했던 반복형에 iter()를 호출해야 함
    - 반복자 자체에 iter()를 호출하는 것은 소용 없은 
        - Iterator.\__iter\__() 은 단지 self를 반환하도록 구현되어서 소진된 반복자 재설정하지 못함

## 14.3 Sentence 버전 2: 고전적인 반복자

#### 예제 14-4 sentence_iter.py : 반복자 패턴을 이용한 Sentence 구현

In [2]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return "Sentence(''%s')" % reprlib.repr(self.text)
    
    def __iter__(self): # 앞에 구현한 class에 추가 구현
        return SentenceIterator(self.words) # iter가 반복형 객체를 생성해서 반환함으로써 반복형 프로토콜 완전히 구현
    
    
class SentenceIterator:
    
    def __init__(self, words):
        self.words = words # 단어 리스트에 대한 참조
        self.index = 0 # 다음에 가져올 단어 결정
        
    def __next__(self):
        
        try:
            words = self.words[self.index]
        except IndexError:
            raise StopIteration()
        self.index += 1
        return word
    
    def __iter__(self):
        return self

- 이 예제가 작동하는데 SentenceIterator의 \__iter\__()을 구현할 필요는 없지만 이 것이 올바른 구현
- 반복자는 \__next\__()와 \__iter\__() 메서드를 모두 구현해야하고 이를 둘다 구현하면 

In [11]:
issubclass(SentenceIterator, abc.Iterator)

True

- SentenceIterator를 abc.Iterator에서 상속하면 구상 메서드인 abc.Iterator.\__iter\__() 상속 받을 수 있음


### 14.3.1 Sentence를 반복자로 만들기 : 좋지 않은 생각
- 반복형과 반복자를 만드는데 흔히 발생하는 오류는 둘을 혼동하기 때문
- 반복형
    - 호출될 때마나 반복자를 새로 생성하는 \__iter\__() 메서드를 가지고 있음
- 반복자
    - 개별 항목을 반환하는 \__next\__() 메서드와 self를 반환하는 \__iter\__()메서드를 가지고 있음
- 반복자는 반복형이지만, 반복형은 반복자가 아님

## 14.4 Sentence 버전 3: 제너레이터 함수

#### 예제 14-5 sentence_gen.py: 제너레이터 함수를 사용해서 구현한 Sentence

In [12]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence:
    
    def __init__(self, text):
        self.text = text
        self.words = RE_WORD.findall(text)
        
    def __repr__(self):
        return "Sentence(''%s')" % reprlib.repr(self.text)
    
    def __iter__(self): # 앞에 구현한 class에 추가 구현
        for word in self.words:
            yield word # 현재 단어를 생성
        return # 함수가 끝에 도달하면 값을 자동으로 반환하므로, 이 return 문은 필요 없음
        # 제너레이터 함수는 StopIteration을 발생시키지 않고 값을 모두 생성한 후 그냥 빠져나감
    
# 완료, 별도의 반복자 클래스가 필요 없음

- 위의 예제에서 반복자는 사실 제너레이터 객체로서, \__iter\__() 메서드 호출할 때 자동으로 생성된

### 14.4.1 제너레이터 함수의 작동 방식
- 본체 안에 yield 키워드를 가진 함수는 모두 제너레이터 함수
- 제너레이터 함수는 호출되면 제너레이터 객체를 반환
    - 즉, 제너레이터 함수는 제너레이터 팩토리

In [17]:
def gen_123(): # yield 키워드를 포함하고 있는 함수는 모두 제너레이터 함수
    yield 1 
    yield 2
    yield 3
    
print("gen_123 : ", gen_123) # 함수 객체임을 확인 
print("gen_123() : ", gen_123()) # 호출하면 제너레이터 객체 반환
for i in gen_123(): 
    print(i)
    
g = gen_123() # 제너레이터 객체를 g에 할당
next(g) # g가 반복자이기도 하므로 next(g)로 호출하면 yield가 다음 항목을 가지고 옴
next(g)
next(g)
next(g) # 함수 본체 실행이 완료되면 제너레이터 객체는 StopIteration 발생

gen_123 :  <function gen_123 at 0x000001A848E78B88>
gen_123() :  <generator object gen_123 at 0x000001A848B91548>
1
2
3


StopIteration: 

- 제너레이터 함수를 함수 본체를 포함하는 제너레이터 객체를 생성
- next()를 제너레이터 객체에 호출하면 함수 본체에 있는 다음 yield로 진행
- next()는 함수 본체가 중단된 곳에서 생성된 값을 평가
- 함수 본체가 반환될 때, 이 함수를 포함하고 있는 제너레이터 객체는 Iterator 프로토콜에 따라 StopIteration 예외 발생

#### 예제 14-6 실행할 때 메시지를 출력하는 제너레이터 함수

In [19]:
def gen_AB(): # 제너레이터 함수는 여느 함수와 동일하게 정의되지만 yield 키워드 사용
    print("start") 
    yield 'A' # for문에서 처음 next()를 암묵적으로 호출하면 'start' 출력하고, 첫번째 yield 문에서 'A' 출력
    print("continue")
    yield 'B' # for 루프에서 두번째 next()를 암묵적으로 호출하면 continue를 출력하고 'B' 생성
    print("end.") # 세 번째 next()가 호출되면 'end'를 출력하고 함수 본체의 끝까지 실행되어, 제너레이터 객체가 StopIteration 예외 발생

for c in gen_AB(): # 반복하기 위해 for 루프는 g = iter(gend_AB())와 대등한 문장을 실행해서 제너레이터 객체 가져오고, 
                    # 반복할 때마다 next(g) 호출
    print("-->", c)

start
--> A
continue
--> B
end.


- for 루프가 소비할 값 'A'를 생성하여, 이 값을 변수 c에 할당
- for 루프는 제너레이터 객체가 발생하는 StopIteration 예외를 잡은 후 루프 종료

## 14.5 Sentence 버전 4: 느긋한 구현

#### 예제 14-7 sentence_gen2.py : re.finditer() 제너레이터 함수를 호출하는 생성자 함수를 이요해서 구현한 Sentence

In [20]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence:
    
    def __init__(self, text):
        self.text = text
      
        
    def __repr__(self):
        return "Sentence(''%s')" % reprlib.repr(self.text)
    
    def __iter__(self): # 앞에 구현한 class에 추가 구현
        for match in RE_WORD.finditer(self.text): # self.text에서 RE_WORD에 대응되는 단어들의 반복자인 MatchObject 객체 생성
            yield match.group() # MatchObject 객체에 매칭되는 텍스트 추출

## 14.6 Sentence 버전 : 제너레이터 표현식

#### 예제 14-8 지능형 리스트와 제너레이터 표현식에 사용된 gen_AB() 제너레이터 함수

In [23]:
def gen_AB(): # 제너레이터 함수는 여느 함수와 동일하게 정의되지만 yield 키워드 사용
    print("start") 
    yield 'A' # for문에서 처음 next()를 암묵적으로 호출하면 'start' 출력하고, 첫번째 yield 문에서 'A' 출력
    print("continue")
    yield 'B' # for 루프에서 두번째 next()를 암묵적으로 호출하면 continue를 출력하고 'B' 생성
    print("end.") # 세 번째 next()가 호출되면 'end'를 출력하고 함수 본체의 끝까지 실행되어, 제너레이터 객체가 StopIteration 예외 발생


In [34]:
res1 = [x*3 for x in gen_AB()]

start
continue
end.


In [35]:
res1

['AAA', 'BBB']

In [36]:
for i in res1:
    print("-->", i)

--> AAA
--> BBB


In [37]:
res2 = (x*3 for x in gen_AB()) # gen_AB()를 호출하지만 여기에서 소비하지 않음

In [38]:
res2 # 제너레이터 객체

<generator object <genexpr> at 0x000001A8488D80C8>

In [39]:
for i in res2:
    print("-->", i)

start
--> AAA
continue
--> BBB
end.


<hr>

#### 예제 14-9 sentence_genex.py : 제너레이터 표현식을 사용한 Sentence

In [41]:
import re
import reprlib

RE_WORD = re.compile("\w+")

class Sentence:
    
    def __init__(self, text):
        self.text = text
      
        
    def __repr__(self):
        return "Sentence(''%s')" % reprlib.repr(self.text)
    
    def __iter__(self): 
        return (match.group for match in RE_WORD.finditer(self.text))

## 14.7 제너레이터 표현식: 언제 사용하나?

- 제너레이터 표현식 
    - 논리가 간단한 경우
- 제너레이터 함수
    - 표현식이 여러 줄에 걸쳐있는 경우